In [1]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd
from collections import Counter
from patsy.contrasts import ContrastMatrix
import numpy as np
# import networkx as nx

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')


### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

In [2]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

dataset_ids = [x.split('/')[-1]for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_sst_roi_timelockstopsignal', 'subject_level_model/*')))]
dataset_ids = [x for x in dataset_ids if not 'GdH' in x]

model_ns = ['0']

if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model
if model_ns[0] == '2':
    contrasts = ['0','1','2','3','4','5']  # task from second level model

subject_ids = [x.split('/')[-1].split('-')[-1] for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_sst_roi', 'subject_level_model', '*', 'sub-*'))) if not 'GdH' in x]

work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', 'all_datasets_roi_M1preSMA')
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_sst_roi', 'subject_level_model')

# template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')
template_brain_mask = '/home/scotti/projects/3t_7t_sst_comparison/derivatives/mni3mm_brain_mask.nii.gz' #os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_sst_roi_M1preSMA_timelockstopsignal", "group_level_model", 'all_datasets', f'model-{model_ns[0]}')

templates = {'level2_cope': os.path.join(slm_folder, '*[!_GdH]','sub-*', 'func', 'model-{model_n}', '*task-s*_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, '*[!_GdH]','sub-*', 'func', 'model-{model_n}', '*task-s*_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, '*[!_GdH]','sub-*', 'func', 'model-{model_n}', '*task-s*_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}


In [3]:
print(f"""
datasets : {dataset_ids}
model_n : {model_ns}
no. subs : {len(subject_ids)}
subs : {subject_ids}
work_dir : {work_dir}
slm folder : {slm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level2_cope'].format(dataset_id='*',subject_id='*',model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level2_varcope'].format(dataset_id='*',subject_id='*',model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level2_tdof'].format(dataset_id='*',subject_id='*',model_n=model_ns[0],contrast_n=0)))}
""")


datasets : ['Leipzig_7T_SM', 'NTNU_7T_SJSI', 'aron_3T', 'openfmri_3T']
model_n : ['0']
no. subs : 157
subs : ['01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '13', '15', '16', '17', '18', '002', '003', '005', '006', '007', '009', '011', '012', '014', '015', '016', '017', '018', '020', '021', '023', '024', '025', '026', '029', '031', '032', '033', '034', '035', '037', '038', '039', '040', '041', '042', '01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12', '13', '14', '15', '10159', '10171', '10206', '10217', '10228', '10235', '10249', '10273', '10274', '10280', '10290', '10292', '10304', '10316', '10321', '10325', '10329', '10339', '10340', '10345', '10347', '10356', '10361', '10365', '10388', '10429', '10438', '10440', '10448', '10455', '10460', '10471', '10478', '10487', '10492', '10506', '10517', '10523', '10524', '10525', '10557', '10565', '10570', '10575', '10624', '10629', '10631', '10638', '10668', '10674', '10680', '10686', '10692', '10697', '10704', '10

In [4]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [5]:
##
workflow = pe.Workflow(name='feat_level3_sst_roi_M1preSMA_alldatasets_timelockstopsignal')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n']), name='identity') # , 'fwhm'
identity.iterables = [('contrast_n', contrasts),
#                       ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
# workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [6]:
# get dataset + subject ids
def get_subject_ids():
    import glob
    import os
    all_dirs = sorted(glob.glob(os.path.join('/home/scotti/projects/3t_7t_sst_comparison', 'derivatives', 'glm_feat_sst_roi_timelockstopsignal', 'subject_level_model', '*', 'sub-*')))
    all_dirs = [x for x in all_dirs if not '_GdH' in x]
#     all_dirs = sorted(glob.glob(os.path.join('/home/scotti/projects/3t_7t_sst_comparison', 'derivatives', 'glm_feat_sst_roi', 'subject_level_model', '*', 'sub-*')))
#     all_dirs = [x for x in all_dirs if not '_GdH' in x]
    
    dataset_ids, subject_ids = zip(*[(x.split('/')[-2],x.split('/')[-1].split('-')[-1]) for x in all_dirs]) #zip(*[(x.split('/')[-2],x.split('/')[-1].split('-')[-1]) for x in sorted(glob.glob(os.path.join('/home/scotti/projects/3t_7t_sst_comparison', 
#                             'derivatives', 'glm_feat_sst_roi', 'subject_level_model', '*', 'sub-*')))])
    
    return dataset_ids, subject_ids

# define node to output subjects
subject_id_getter = pe.Node(util.Function(output_names=['dataset_ids','subject_ids'],
                                          function=get_subject_ids),
                                          name='subject_id_getter')

# create design matrix witih dummy variables for the different datasets (experimental)
def get_design_matrix(second_level_contrast, dataset_ids, subject_ids): #=subject_ids):
    print(second_level_contrast)
    
    import numpy as np
    from patsy.contrasts import ContrastMatrix
    from collections import Counter
    
#     # code to generate correct simple dummy variables
#     def _name_levels(prefix, levels):
#         return ["[%s%s]" % (prefix, level) for level in levels]

#     class Simple(object):
#         def _simple_contrast(self, levels):
#             nlevels = len(levels)
#             contr = -1.0 / nlevels * np.ones((nlevels, nlevels - 1))
#             contr[1:][np.diag_indices(nlevels - 1)] = (nlevels - 1.0) / nlevels
#             return contr

#         def code_with_intercept(self, levels):
#             contrast = np.column_stack(
#                 (np.ones(len(levels)), self._simple_contrast(levels))
#             )
#             return ContrastMatrix(contrast, _name_levels("Simp.", levels))

#         def code_without_intercept(self, levels):
#             contrast = self._simple_contrast(levels)
#             return ContrastMatrix(contrast, _name_levels("Simp.", levels[:-1]))
        
    levels = [1,2,3,4]
#     contrast = Simple().code_without_intercept(levels)
    
#     dummies = [[-1,-1,-1,-1],[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
#     dummies = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]
    dummies = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
    
    # for regressors do intercept and dummy variables
    regressors = {#'intercept': [1 for x in range(len(subject_ids))],
#                   'GdH': list(np.repeat([item[0] for item in dummies],list(Counter(dataset_ids).values()))),
                  'SM': list(np.repeat([item[0] for item in dummies],list(Counter(dataset_ids).values()))),
                  'SJSI': list(np.repeat([item[1] for item in dummies],list(Counter(dataset_ids).values()))),
                  'aron': list(np.repeat([item[2] for item in dummies],list(Counter(dataset_ids).values()))),
                  'openfmri': list(np.repeat([item[3] for item in dummies],list(Counter(dataset_ids).values())))}
    
    # contrasts (3rd-level), original way, this way calculates difference of each group and mean.
#     third_level_contrasts = [('Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 0, 0, 0, 0]),
#                              ('-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 0, 0]),
#                              ('SM-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 1.0, 0, 0, 0]),
#                              ('SJSI-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 1.0, 0, 0]),
#                              ('aron-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 1.0, 0]),
#                              ('openfmri-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 0, 1.0])
#                             ]
    
    # new method to calculate exact means so do not have to reconstruct later
    third_level_contrasts = [('Group mean', 'T', ['SM', 'SJSI', 'aron', 'openfmri'], [1.0, 1.0, 1.0, 1.0]),
                             ('-Group mean', 'T', ['SM', 'SJSI', 'aron', 'openfmri'], [-1.0, -1.0, -1.0, -1.0]),
#                              ('GdH mean', 'T', ['GdH', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 0.0, 0.0, 0.0, 0.0]),
                             ('SM mean', 'T', ['SM', 'SJSI', 'aron', 'openfmri'], [01.0, 0, 0, 0]),
                             ('SJSI mean', 'T', ['SM', 'SJSI', 'aron', 'openfmri'], [0, 1.0, 0, 0]),
                             ('aron mean', 'T', ['SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 1.0, 0]),
                             ('openfmri mean', 'T', ['SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 1.0])
                            ]

# Unsure which third level contrasts to use ?? fix
#     third_level_contrasts = [('Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [1.0, 0, 0, 0, 0]),
#                              ('-Group mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [-1.0, 0, 0, 0, 0]),
#                              ('SM mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 1.0, 0, 0, 0]),
#                              ('-SM mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, -1.0, 0, 0, 0]),
#                              ('SJSI mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 1.0, 0, 0]),
#                              ('-SJSI mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, -1.0, 0, 0]),
#                              ('aron mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 1.0, 0]),
#                              ('-aron mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, -1.0, 0]),
#                              ('openfmri mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 0, 1.0]),
#                              ('-openfmri mean', 'T', ['intercept', 'SM', 'SJSI', 'aron', 'openfmri'], [0, 0, 0, 0, -1.0]),
#                              ]
    
    # fix this so can use different vairrance group: issue is that design matrix is not separable
    groups = list(np.repeat(levels, list(Counter(dataset_ids).values()))) # set different dataset as groups to estimate variance separately
#     groups = [1 for x in range(len(subject_ids))]
    
    return third_level_contrasts, regressors, groups

contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'dataset_ids', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors', 'groups'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')

workflow.connect(subject_id_getter, 'dataset_ids', contrastgen_l3, 'dataset_ids')
workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')
workflow.connect(contrastgen_l3, 'groups', level3model, 'groups')


In [7]:
# run flame 1 + 2
flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [8]:
# ## cluster thresholding
# # Smoothness estimation
# smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
# smoothestimate.inputs.mask_file = template_brain_mask

# workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# # get volume
# get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
# get_volume.inputs.in_file = template_brain_mask

# # Cluster threshold
# grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
# grf_cluster.iterables = [("threshold", [2.3, 3.1])]
# grf_cluster.inputs.out_localmax_txt_file = True
# grf_cluster.inputs.pthreshold = 0.05
# grf_cluster.inputs.out_index_file = True
# grf_cluster.inputs.out_threshold_file = True

# def volume_convert(input):
#     return int(input[0])

# workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
# workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
# workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [9]:
## Datasink, how to output the data
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir

# rename some files so they're not split over a bunch of folders
# depending on the FWHM used, the regexp might have to be adpated due to number of elements
substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                         'model-\\2/model-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\7_flame-\\3_desc-\\6_voxelthreshold-\\4.nii.gz'),
                        (r'third_level_model/level3_.*/_contrast_n_(\d+)_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                          'model-\\2/model-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\5_flame-\\3_desc-\\4.nii.gz'),
                        # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                        #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                       ]

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [10]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 12, 'memory_gb': 120})

240715-13:47:54,371 nipype.workflow INFO:
	 Workflow feat_level3_sst_roi_M1preSMA_alldatasets_timelockstopsignal settings: ['check', 'execution', 'logging', 'monitoring']
240715-13:47:54,420 nipype.workflow INFO:
	 Running in parallel.
240715-13:47:54,447 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 120.00/120.00, Free processors: 12/12.
240715-13:47:54,481 nipype.workflow INFO:
	 [Job 0] Cached (feat_level3_sst_roi_M1preSMA_alldatasets_timelockstopsignal.subject_id_getter).
240715-13:47:54,482 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level3_sst_roi_M1preSMA_alldatasets_timelockstopsignal.get_volume".
240715-13:47:54,529 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi_M1preSMA_alldatasets_timelockstopsignal.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/all_datasets_roi_M1preSMA/feat_level3_sst_roi_M1preSMA_alldatasets_timelockstopsignal/_contrast_n_0_model_n_0/s

In [11]:
print('done')

done
